# Project 01 
 by Hafsa and Nicole

# How did national murder rates change in 2020 from previous years?

### Below we will explore crime statistics from the Federal Bureau of Investigation to answer this question.

In [10]:
# Dependencies and modules:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from pprint import pprint
import seaborn as sns
import statistics
import math
from scipy import stats

# Import API key
from api_keys import FBI_api

In [6]:
states = {'AK': 'Alaska',
        'AL': 'Alabama',
        'AR': 'Arkansas',
        'AS': 'American Samoa',
        'AZ': 'Arizona',
        'CA': 'California',
        'CO': 'Colorado',
        'CT': 'Connecticut',
        'DC': 'District of Columbia',
        'DE': 'Delaware',
        'FL': 'Florida',
        'GA': 'Georgia',
        'GU': 'Guam',
        'HI': 'Hawaii',
        'IA': 'Iowa',
        'ID': 'Idaho',
        'IL': 'Illinois',
        'IN': 'Indiana',
        'KS': 'Kansas',
        'KY': 'Kentucky',
        'LA': 'Louisiana',
        'MA': 'Massachusetts',
        'MD': 'Maryland',
        'ME': 'Maine',
        'MI': 'Michigan',
        'MN': 'Minnesota',
        'MO': 'Missouri',
        'MP': 'Northern Mariana Islands',
        'MS': 'Mississippi',
        'MT': 'Montana',
        'NA': 'National',
        'NC': 'North Carolina',
        'ND': 'North Dakota',
        'NE': 'Nebraska',
        'NH': 'New Hampshire',
        'NJ': 'New Jersey',
        'NM': 'New Mexico',
        'NV': 'Nevada',
        'NY': 'New York',
        'OH': 'Ohio',
        'OK': 'Oklahoma',
        'OR': 'Oregon',
        'PA': 'Pennsylvania',
        'PR': 'Puerto Rico',
        'RI': 'Rhode Island',
        'SC': 'South Carolina',
        'SD': 'South Dakota',
        'TN': 'Tennessee',
        'TX': 'Texas',
        'UT': 'Utah',
        'VA': 'Virginia',
        'VI': 'Virgin Islands',
        'VT': 'Vermont',
        'WA': 'Washington',
        'WI': 'Wisconsin',
        'WV': 'West Virginia',
        'WY': 'Wyoming'}

In [7]:
# base_url = 'http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID='+ api_key
# city = 'st. louis'

# req = requests.get(base_url + f'&q={city}')
# print()

# #create a json output of the data

# json_data = req.json()
# print(json_data)           <-- This is the code block that we used in Week 5. I copied it here to use as a template.



# Endpoint: /api/summarized/query            <---Here are all parts needed to build HTTP,
# Base URL: api.usa.gov/crime/fbi/sapi/          I just can't figure out the proper order and syntax.
# My api: FBI_api    

    
base_url = 'https://api.usa.gov/crime/fbi/sapi/api/estimates/national/2010/2020?API_KEY=' + FBI_api

#

req = requests.get(base_url)
print(req)  # response 200!

json_data = req.json()
print(json_data)

<Response [200]>
{'results': [{'state_id': None, 'state_abbr': '', 'year': 2015, 'population': 320896618, 'violent_crime': 1199310, 'homicide': 15883, 'rape_legacy': 91261, 'rape_revised': 126134, 'robbery': 328109, 'aggravated_assault': 764057, 'property_crime': 8024115, 'burglary': 1587564, 'larceny': 5723488, 'motor_vehicle_theft': 713063, 'arson': 44139}, {'state_id': None, 'state_abbr': '', 'year': 2010, 'population': 309330219, 'violent_crime': 1251248, 'homicide': 14722, 'rape_legacy': 85593, 'rape_revised': None, 'robbery': 369089, 'aggravated_assault': 781844, 'property_crime': 9112625, 'burglary': 2168459, 'larceny': 6204601, 'motor_vehicle_theft': 739565, 'arson': 56005}, {'state_id': None, 'state_abbr': '', 'year': 2011, 'population': 311587816, 'violent_crime': 1206031, 'homicide': 14661, 'rape_legacy': 84175, 'rape_revised': None, 'robbery': 354772, 'aggravated_assault': 752423, 'property_crime': 9052743, 'burglary': 2185140, 'larceny': 6151095, 'motor_vehicle_theft': 716

In [4]:
#data we want: year, homicide, population, state, state abr
#make a null set and append from api
murder_by_state= {'State ID':[], 'State Abbreviation': [], 'Year': [], 'Population': [], 'Number of Homicides': []}

for state_abr in states:
   base_url = "https://api.usa.gov/crime/fbi/sapi/api/estimates/states/{StateAbbr}/2010/2020?API_KEY=" + FBI_api   #how to add the state abbr in here?
    
    try:
        
        req = requests.get(base_url + f'&q={state_abr}')
        
        data = req.json()

        murder_by_state['State ID'].append(data['state_id'])
        murder_by_state['State Abbreviation'].append(data['state_abbr'])
        murder_by_state['Year'].append(data['year'])
        murder_by_state['Population'].append(data['population'])
        murder_by_state['Number of Homicides'].append(data['homicide'])
        
 
        print("processing " + state_abr)
    except:
        print(state_abr+ " not found")                    
        pass

AK not found
AL not found
AR not found
AS not found
AZ not found
CA not found
CO not found
CT not found
DC not found
DE not found
FL not found
GA not found
GU not found
HI not found
IA not found
ID not found
IL not found
IN not found
KS not found
KY not found
LA not found
MA not found
MD not found
ME not found
MI not found
MN not found
MO not found
MP not found
MS not found
MT not found
NA not found
NC not found
ND not found
NE not found
NH not found
NJ not found
NM not found
NV not found
NY not found
OH not found
OK not found
OR not found
PA not found
PR not found
RI not found
SC not found
SD not found
TN not found
TX not found
UT not found
VA not found
VI not found
VT not found
WA not found
WI not found
WV not found
WY not found


In [6]:
murder_by_state

{'State ID': [],
 'State Abbreviation': [],
 'Year': [],
 'Population': [],
 'Number of Homicides': []}

<Response [200]>
{'results': [{'state_id': 56, 'state_abbr': 'WY', 'year': 2017, 'population': 578934, 'violent_crime': 1358, 'homicide': 14, 'rape_legacy': None, 'rape_revised': 253, 'robbery': 98, 'aggravated_assault': 993, 'property_crime': 10528, 'burglary': 1556, 'larceny': 8211, 'motor_vehicle_theft': 761, 'arson': 48}, {'state_id': 56, 'state_abbr': 'WY', 'year': 2010, 'population': 564554, 'violent_crime': 1117, 'homicide': 8, 'rape_legacy': 162, 'rape_revised': None, 'robbery': 77, 'aggravated_assault': 870, 'property_crime': 13869, 'burglary': 2151, 'larceny': 11126, 'motor_vehicle_theft': 592, 'arson': 71}, {'state_id': 56, 'state_abbr': 'WY', 'year': 2011, 'population': 567356, 'violent_crime': 1245, 'homicide': 18, 'rape_legacy': 146, 'rape_revised': None, 'robbery': 71, 'aggravated_assault': 1010, 'property_crime': 12878, 'burglary': 1864, 'larceny': 10493, 'motor_vehicle_theft': 521, 'arson': 58}, {'state_id': 56, 'state_abbr': 'WY', 'year': 2012, 'population': 576626, '